**LEITURA DOS DADOS DO SUSTAIN BENCH**

In [88]:
import numpy as np
import pandas as pd

# -----------------------------
# Função para carregar um .npz
# -----------------------------
def load_npz_array(path):
    data = np.load(path, allow_pickle=True)
    return data[data.files[0]]

# -----------------------------
# 1. Carregar todos os arquivos de treino
# -----------------------------
X_hist = load_npz_array("train_hists.npz")
X_ndvi = load_npz_array("train_ndvi.npz")
X_loc = load_npz_array("train_locs.npz")
X_year = load_npz_array("train_years.npz")
X_key = load_npz_array("train_keys.npz")
y = load_npz_array("train_yields.npz")

# -----------------------------
# 2. Mostrar shapes
# -----------------------------
print("Shapes dos arrays:")
print("Hist:", X_hist.shape)
print("NDVI:", X_ndvi.shape)
print("Locs:", X_loc.shape)
print("Years:", X_year.shape)
print("Keys:", X_key.shape)
print("Yields:", y.shape)

# -----------------------------
# 3. Montar um DataFrame para visualização
# -----------------------------

# Hist tem muitas colunas → renomear
X_hist_reshaped = X_hist.reshape(X_hist.shape[0], -1)
hist_cols = [f"hist_{i}" for i in range(X_hist_reshaped.shape[1])]
df_hist = pd.DataFrame(X_hist_reshaped, columns=hist_cols)

# NDVI também
ndvi_cols = [f"ndvi_{i}" for i in range(X_ndvi.shape[1])]
df_ndvi = pd.DataFrame(X_ndvi, columns=ndvi_cols)

# Locs - CORRIGIDO: cria duas colunas separadas
df_loc = pd.DataFrame({'lat_original': [None]*len(X_loc), 'lon_original': [None]*len(X_loc)})

df_year = pd.DataFrame(X_year, columns=["year"])
df_key = pd.DataFrame(X_key, columns=["region_id"])
df_y = pd.DataFrame(y, columns=["yield"])

# Junta tudo
df = pd.concat([df_hist, df_ndvi, df_loc, df_year, df_key, df_y], axis=1)

print("\nPrimeiras linhas:")
print(df.head())

print("\nColunas totais:")
print(df.columns)

Shapes dos arrays:
Hist: (247, 32, 32, 9)
NDVI: (247, 32)
Locs: (247,)
Years: (247,)
Keys: (247,)
Yields: (247,)

Primeiras linhas:
     hist_0    hist_1    hist_2   hist_3    hist_4    hist_5    hist_6  \
0  0.000000  0.000270  0.000025  0.00000  0.000037  0.000061  0.000061   
1  0.000000  0.000945  0.000000  0.00000  0.000065  0.000126  0.000189   
2  0.000673  0.000657  0.000000  0.00003  0.000547  0.000437  0.002556   
3  0.000000  0.000538  0.000107  0.00000  0.000401  0.000000  0.000000   
4  0.000000  0.000343  0.000000  0.00000  0.000144  0.000274  0.000480   

   hist_7  hist_8    hist_9  ...   ndvi_27   ndvi_28   ndvi_29   ndvi_30  \
0     0.0     0.0  0.000000  ...  0.698605  0.640461  0.566693  0.645517   
1     0.0     0.0  0.000000  ...  0.822922  0.786009  0.720077  0.574686   
2     0.0     0.0  0.000485  ...  0.637951  0.673795  0.624429  0.531740   
3     0.0     0.0  0.000000  ...  0.729741  0.787198  0.824194  0.805108   
4     0.0     0.0  0.000000  ...  0.699269 

# **ACHAR INFORMAÇOES ESPACIAIS**

O conjunto de dados carece de Latitude e Longitude, o que é um impedimento crucial.

A Solução é: Derivar o Lat/Long do REGION_ID usando o nome do município e a ferramenta Geopy.

A Justificativa Principal é: Embora as features existentes já sejam climáticas históricas, a adição do Lat/Long não só oferece um contexto geográfico direto, mas, mais importante, serve como chave de agregação para incorporar dados climáticos externos mais ricos e detalhados (ex: interpolação de dados do INMET), maximizando assim o poder preditivo do modelo.

In [89]:
# Instala a biblioteca geopy para geocodificação
!pip install geopy
!pip install unidecode

Para melhorar a velocidade e para não ter o IP banido pela API, baixamos um .csv com a lat/long de todas as cidades brasileiras

In [90]:
import pandas as pd

# Baixa o arquivo BR completo do GeoNames
url = "https://download.geonames.org/export/dump/BR.zip"
zip_path = "BR.zip"

!wget -q {url} -O {zip_path}
!unzip -o BR.zip

# O arquivo útil chama-se BR.txt
cols = [
    "geonameid", "name", "asciiname", "alternatenames",
    "latitude", "longitude", "feature_class", "feature_code",
    "country_code", "cc2", "admin1_code", "admin2_code",
    "admin3_code", "admin4_code", "population", "elevation",
    "dem", "timezone", "modification_date"
]

df_geonames = pd.read_csv("BR.txt", sep="\t", header=None, names=cols, dtype=str)

# Filtrar apenas cidades oficiais (PPL = populated place)
df_cities = df_geonames[df_geonames["feature_code"] == "PPLA"]  # capitais municipais
df_villages = df_geonames[df_geonames["feature_code"] == "PPL"]  # municípios normais
df_all = pd.concat([df_cities, df_villages])

# Selecionar só o que importa
df_final = df_all[["name", "latitude", "longitude"]].copy()

# Converter para float
df_final["latitude"] = df_final["latitude"].astype(float)
df_final["longitude"] = df_final["longitude"].astype(float)

# Salvar CSV
df_final.to_csv("municipios_brasil_lat_lon.csv", index=False)

print("Municípios carregados:")
print(df_final.head())

Archive:  BR.zip
  inflating: readme.txt              
  inflating: BR.txt                  
Municípios carregados:
           name  latitude  longitude
1688   Teresina  -5.08917  -42.80194
3552   São Luís  -2.52972  -44.30278
5940     Recife  -8.05389  -34.88111
9199      Natal  -5.79500  -35.20944
11154    Maceió  -9.66583  -35.73528


In [91]:
import pandas as pd
from difflib import get_close_matches
import unicodedata

# Carrega o CSV com as coordenadas dos municípios
municipios_coords = pd.read_csv("municipios_brasil_lat_lon.csv")

# Função para normalizar texto (remover acentos, minúsculas)
def normalize_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower().strip()
    # Remove acentos
    text = unicodedata.normalize('NFD', text)
    text = ''.join(char for char in text if unicodedata.category(char) != 'Mn')
    return text

# Normaliza os nomes dos municípios no CSV
municipios_coords['name_normalized'] = municipios_coords['name'].apply(normalize_text)

# Cria um dicionário para busca rápida
coords_dict = dict(zip(
    municipios_coords['name_normalized'],
    zip(municipios_coords['latitude'], municipios_coords['longitude'])
))

def get_lat_lon(region_id):
    try:
        # Extrai o nome da região (removendo o ano e '_brasil_')
        region_name = region_id.split('_brasil_')[0].replace('_', ' ').strip()
        region_normalized = normalize_text(region_name)

        # Estratégia 1: Busca exata
        if region_normalized in coords_dict:
            return coords_dict[region_normalized]

        # Estratégia 2: Pega a última palavra (geralmente é o município)
        last_word = region_normalized.split()[-1] if region_normalized else ""
        if last_word and last_word in coords_dict:
            return coords_dict[last_word]

        # Estratégia 3: Testa cada palavra individualmente
        words = region_normalized.split()
        for word in reversed(words):  # começa do fim (mais específico)
            if word in coords_dict:
                return coords_dict[word]

        # Estratégia 4: Busca aproximada (fuzzy matching)
        matches = get_close_matches(region_normalized, coords_dict.keys(), n=1, cutoff=0.8)
        if matches:
            return coords_dict[matches[0]]

        # Estratégia 5: Busca aproximada com a última palavra
        if last_word:
            matches = get_close_matches(last_word, coords_dict.keys(), n=1, cutoff=0.7)
            if matches:
                return coords_dict[matches[0]]

        return None, None

    except Exception as e:
        print(f"Erro ao geocodificar {region_id}: {e}")
        return None, None

# Aplica a função para obter as coordenadas
df['new_lat'], df['new_lon'] = zip(*df['region_id'].apply(get_lat_lon))

print("Valores únicos de new_lat e new_lon (amostra):")
print(df[['region_id', 'new_lat', 'new_lon']].head(10))
print(f"\nNúmero de NAs em new_lat: {df['new_lat'].isnull().sum()}")
print(f"Total de registros: {len(df)}")

# Debug: mostra os que falharam
print("\n=== Exemplos que falharam ===")
failed = df[df['new_lat'].isnull()]['region_id'].head(10)
for region in failed:
    region_name = region.split('_brasil_')[0].replace('_', ' ').strip()
    print(f"'{region_name}'")

Valores únicos de new_lat e new_lon (amostra):
                                     region_id   new_lat   new_lon
0  triangulo mineiroalto paranaiba_brasil_2008 -19.67722 -51.19083
1                            assis_brasil_2011  -5.00600 -40.84601
2                       sul goiano_brasil_2007 -21.53722 -43.20167
3    centro ocidental riograndense_brasil_2011 -27.23224 -52.02768
4            oriental do tocantins_brasil_2011  -3.70757 -45.30524
5            noroeste riograndense_brasil_2016 -10.66140 -38.99077
6                 oeste paranaense_brasil_2006  -2.41062 -48.05416
7   sudoeste de mato grosso do sul_brasil_2012 -12.95000 -39.26667
8                     leste goiano_brasil_2014 -26.90393 -53.55319
9             extremo oeste baiano_brasil_2012  -6.29253 -62.23404

Número de NAs em new_lat: 0
Total de registros: 247

=== Exemplos que falharam ===


In [92]:
# Atualiza as colunas 'lat' e 'lon' com as novas coordenadas
df['lat'] = df['new_lat']
df['lon'] = df['new_lon']

# Remove as colunas temporárias 'new_lat' e 'new_lon'
df = df.drop(columns=['new_lat', 'new_lon'])

print("Colunas 'lat' e 'lon' atualizadas. Primeiras linhas:")
print(df[['lat', 'lon']].head())
print(f"\nNúmero de NAs restantes em lat: {df['lat'].isnull().sum()}")
print(f"Número de NAs restantes em lon: {df['lon'].isnull().sum()}")

Colunas 'lat' e 'lon' atualizadas. Primeiras linhas:
        lat       lon
0 -19.67722 -51.19083
1  -5.00600 -40.84601
2 -21.53722 -43.20167
3 -27.23224 -52.02768
4  -3.70757 -45.30524

Número de NAs restantes em lat: 0
Número de NAs restantes em lon: 0


# Implementaçao do Modelo

In [93]:
!pip install scikit-learn
!pip install streamlit pyngrok

In [94]:
from pyngrok import ngrok

# Coloque seu token aqui
ngrok.set_auth_token("36RRGdxtdsr6l1S2mMDLnCrDNf2_2pxhtaKfNrdsZJAUEM8oq")

In [95]:
%%writefile app.py
import streamlit as st

st.title("Teste Streamlit no Colab")
st.write("Se você está vendo isso, funcionou!")


Overwriting app.py


In [96]:
df.to_pickle("dados.pkl")


In [97]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

st.title("Modelos de Predição de Produtividade de Soja")

# --------------------------
# Carregar DataFrame salvo
# --------------------------
@st.cache_data
def load_data():
    return pd.read_pickle("dados.pkl")

df = load_data()
st.write("Dimensões do dataset:", df.shape)
st.write(df.head())

# --------------------------
# 1. Seleção do target
# --------------------------
available_targets = ["yield"]
target_col = st.selectbox("Selecione o target", available_targets, index=0)

X = df.drop(columns=[target_col, 'region_id'])
y = df[target_col]

test_size = st.slider("Tamanho do conjunto de teste (%)", 10, 50, 20) / 100
random_state = st.number_input("Random State", value=42)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=random_state
)

# --------------------------
# 2. Seleção do Modelo
# --------------------------
st.subheader("Escolha o Algoritmo")

model_name = st.selectbox(
    "Modelo",
    ["Linear Regression", "Random Forest", "Gradient Boosting"]
)

# Hiperparâmetros
if model_name == "Random Forest":
    n_estimators = st.slider("Número de árvores", 50, 500, 200)
    max_depth = st.slider("Profundidade máxima", 2, 30, 10)
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

elif model_name == "Gradient Boosting":
    learning_rate = st.slider("Learning Rate", 0.01, 0.5, 0.1)
    n_estimators = st.slider("Número de estimadores", 50, 500, 150)
    model = GradientBoostingRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators
    )

else:
    model = LinearRegression()

# --------------------------
# 3. Treinar modelo
# --------------------------
if st.button("Treinar Modelo"):
    with st.spinner("Treinando..."):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Métricas
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)

        st.success("Treinamento Concluído!")

        st.subheader("Métricas de Avaliação")
        st.metric("MAE", f"{mae:.3f}")
        st.metric("RMSE", f"{rmse:.3f}")
        st.metric("R²", f"{r2:.3f}")

        # --------------------------
        # 4. Real vs Predito
        # --------------------------
        st.subheader("Real vs Predito (Linha)")
        df_plot = pd.DataFrame({"Real": y_test.values, "Predito": y_pred})
        st.line_chart(df_plot)

        # Scatter plot
        st.subheader("Dispersão: Real vs Predito")
        fig, ax = plt.subplots()
        ax.scatter(y_test, y_pred, alpha=0.6)
        ax.set_xlabel("Real")
        ax.set_ylabel("Predito")
        ax.grid(True)
        st.pyplot(fig)

        # --------------------------
        # 5. Erros (Resíduos)
        # --------------------------
        st.subheader("Distribuição dos Erros (Resíduos)")
        errors = y_test - y_pred
        fig2, ax2 = plt.subplots()
        ax2.hist(errors, bins=30)
        ax2.set_xlabel("Erro (Real - Predito)")
        ax2.set_ylabel("Frequência")
        st.pyplot(fig2)

        # --------------------------
        # 6. Importância das Features
        # --------------------------
        if model_name in ["Random Forest", "Gradient Boosting"]:
            st.subheader("Importância das Variáveis")

            importances = model.feature_importances_
            feat_df = pd.DataFrame({
                "feature": X.columns,
                "importance": importances
            }).sort_values("importance", ascending=False)

            st.bar_chart(feat_df.set_index("feature"))

            st.write(feat_df)

        # --------------------------
        # 7. Correlação das 10 features mais importantes
        # --------------------------
        st.subheader("Matriz de Correlação (Top 10 Features Importantes)")

        # Só funciona para modelos baseados em árvores
        if model_name in ["Random Forest", "Gradient Boosting"]:
            # Pega as 10 features mais importantes
            feat_df_top10 = feat_df.head(10)
            top10_cols = list(feat_df_top10["feature"])

            # Calcula a correlação apenas entre elas
            corr_top10 = df[top10_cols].corr(numeric_only=True)

            fig3, ax3 = plt.subplots(figsize=(8, 6))
            cax = ax3.matshow(corr_top10)
            fig3.colorbar(cax)

            ax3.set_xticks(range(len(corr_top10.columns)))
            ax3.set_yticks(range(len(corr_top10.columns)))
            ax3.set_xticklabels(corr_top10.columns, rotation=90)
            ax3.set_yticklabels(corr_top10.columns)

            st.pyplot(fig3)

        else:
            st.info("A matriz de correlação baseada em importância só funciona para Random Forest ou Gradient Boosting.")


        # --------------------------
        # 8. Piores e Melhores Previsões
        # --------------------------
        st.subheader("Erros Individuais")
        df_errors = pd.DataFrame({
            "Real": y_test,
            "Predito": y_pred,
            "Erro": errors
        })

        st.write("20 piores erros:")
        st.write(df_errors.sort_values("Erro").head(20))

        st.write("20 melhores acertos:")
        st.write(df_errors.sort_values("Erro", key=abs).head(20))

        # --------------------------
        # 9. Download dos resultados
        # --------------------------
        st.subheader("Download")

        csv = df_errors.to_csv(index=False).encode("utf-8")
        st.download_button(
            label="⬇️ Baixar CSV de Resultados",
            data=csv,
            file_name="resultados_modelo.csv",
            mime="text/csv"
        )


Overwriting app.py


In [ ]:
public_url = ngrok.connect(addr="8050")
print("Acesse aqui:", public_url)

!streamlit run app.py --server.port 8050

Acesse aqui: NgrokTunnel: "https://presecular-yael-unavowably.ngrok-free.dev" -> "http://localhost:8050"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8050
  Network URL: http://172.28.0.12:8050
  External URL: http://34.123.91.119:8050

